<a href="https://colab.research.google.com/github/rianaditro/classification_web_app/blob/main/classification_C5_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirement

In [1]:
import pandas as pd
import numpy as np

In [70]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

from sklearn import tree

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
pip install c45-decision-tree

In [5]:
from C45 import C45Classifier

In [6]:
import seaborn as sns
from matplotlib import pyplot as plt
# generate tree diagram with C45Classifier modul
import graphviz

## Data Pre-processing

In [50]:
df = pd.read_excel('databulu.xlsx')
df.head()

,spesies,texture,structure,color,scale position,scale distance,scale margin,scale pattern,Unnamed: 8,medulla composition,...,medulla margin,hair cross section in the base,hair cross section in the middle,hair cross section in the tip,medulla cross section in the base,medulla cross section in the middle,medulla cross section in the tip,d rambut,d medula,index medula
0,S. barbatus,rough,"straight, curve","black at the base to the middle, broken white ...",transversal,close,rippled,iiregular wave,NaN,amorphous,...,amorphous,circular,circular,circular,flower shape,flower shape,flower shape,35144,28000,0.796722
1,S. barbatus,rough,"straight, curve","black at the base to the middle, broken white ...",transversal,close,rippled,iiregular wave,NaN,amorphous,...,amorphous,circular,circular,circular,flower shape,flower shape,flower shape,36285,29714,0.818906
2,S. barbatus,rough,"straight, curve","black at the base to the middle, broken white ...",transversal,close,rippled,iiregular wave,NaN,amorphous,...,amorphous,circular,circular,circular,flower shape,flower shape,flower shape,40285,33428,0.829788
3,S. barbatus,rough,"straight, curve","black at the base to the middle, broken white ...",transversal,close,rippled,iiregular wave,NaN,amorphous,...,amorphous,circular,circular,circular,flower shape,flower shape,flower shape,31714,26286,0.828845
4,S. barbatus,rough,"straight, curve","black at the base to the middle, broken white ...",transversal,close,rippled,iiregular wave,NaN,amorphous,...,amorphous,circular,circular,circular,flower shape,flower shape,flower shape,39144,31714,0.810188


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 22 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   spesies                              1350 non-null   object 
 1   texture                              1350 non-null   object 
 2   structure                            1350 non-null   object 
 3   color                                1350 non-null   object 
 4   scale position                       1350 non-null   object 
 5   scale distance                       1350 non-null   object 
 6   scale margin                         1350 non-null   object 
 7   scale pattern                        1350 non-null   object 
 8   Unnamed: 8                           0 non-null      float64
 9   medulla composition                  1350 non-null   object 
 10  medulla structure                    1350 non-null   object 
 11  medulla pattern               

In [52]:
# removing empty column
df.dropna(how='all', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 21 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   spesies                              1350 non-null   object 
 1   texture                              1350 non-null   object 
 2   structure                            1350 non-null   object 
 3   color                                1350 non-null   object 
 4   scale position                       1350 non-null   object 
 5   scale distance                       1350 non-null   object 
 6   scale margin                         1350 non-null   object 
 7   scale pattern                        1350 non-null   object 
 8   medulla composition                  1350 non-null   object 
 9   medulla structure                    1350 non-null   object 
 10  medulla pattern                      1350 non-null   object 
 11  medulla margin                

In [13]:
df.nunique()

spesies                                   5
texture                                   1
structure                                 1
color                                     8
scale position                            1
scale distance                            1
scale margin                              1
scale pattern                             1
medulla composition                       1
medulla structure                         1
medulla pattern                           1
medulla margin                            1
hair cross section in the base            3
hair cross section in the middle          3
hair cross section in the tip             2
medulla cross section in the base         3
medulla cross section in the middle       3
medulla cross section in the tip          3
d rambut                               1052
d medula                               1054
index medula                           1342
dtype: int64

In [53]:
# removing low variance feature
ls = df.nunique().to_list()
idx = [i for i in range(len(ls)) if ls[i] == 1]

df= df.drop(df.columns[idx],axis=1)
df.nunique()

spesies                                   5
color                                     8
hair cross section in the base            3
hair cross section in the middle          3
hair cross section in the tip             2
medulla cross section in the base         3
medulla cross section in the middle       3
medulla cross section in the tip          3
d rambut                               1052
d medula                               1054
index medula                           1342
dtype: int64

In [18]:
for col in df.columns:
  if df[col].dtype == 'object':
    print(df[col].value_counts())
    print("=====================================================================")

spesies
S. barbatus       270
S. celebensis     270
S. scrofa         270
S. verrucossus    270
S. scofa          270
Name: count, dtype: int64
color
black at the base to the middle, broken white at the tip                270
black, without gradation                                                270
black at the base, broken white at the middle, dark brown at the tip    270
black at the base and dark brown at the tip                             270
black without gradation                                                  68
red at the base and dark brown at the tip                                68
white at the base and light brown at the tip                             67
dark brown at the base and black at the tip                              67
Name: count, dtype: int64
hair cross section in the base
circular      810
triangular    270
oval          270
Name: count, dtype: int64
hair cross section in the middle
circular      540
oval          540
triangular    270
Name: count, dtype

In [19]:
# notice value in the color columns there is a duplicate for 'black without gradation'
df['color'].value_counts()

color
black at the base to the middle, broken white at the tip                270
black, without gradation                                                270
black at the base, broken white at the middle, dark brown at the tip    270
black at the base and dark brown at the tip                             270
black without gradation                                                  68
red at the base and dark brown at the tip                                68
white at the base and light brown at the tip                             67
dark brown at the base and black at the tip                              67
Name: count, dtype: int64

In [54]:
# cleaning data
df['color'] = df['color'].str.replace(",","")
df['color'].value_counts()

color
black without gradation                                               338
black at the base to the middle broken white at the tip               270
black at the base broken white at the middle dark brown at the tip    270
black at the base and dark brown at the tip                           270
red at the base and dark brown at the tip                              68
white at the base and light brown at the tip                           67
dark brown at the base and black at the tip                            67
Name: count, dtype: int64

In [55]:
# standarization
data_to_scale = df[['d rambut', 'd medula', 'index medula']]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_to_scale)
df[['d rambut', 'd medula', 'index medula']] = scaled_data
df.head()

,spesies,color,hair cross section in the base,hair cross section in the middle,hair cross section in the tip,medulla cross section in the base,medulla cross section in the middle,medulla cross section in the tip,d rambut,d medula,index medula
0,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.553221,-1.276941,0.884385
1,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.543329,-1.255775,1.008958
2,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.508649,-1.209911,1.070064
3,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.582959,-1.298108,1.064772
4,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.518541,-1.231077,0.960003


### Pre-processing for the C5.0

In [56]:
# create a copy of dataframe
tree_df = df
tree_df.head()

,spesies,color,hair cross section in the base,hair cross section in the middle,hair cross section in the tip,medulla cross section in the base,medulla cross section in the middle,medulla cross section in the tip,d rambut,d medula,index medula
0,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.553221,-1.276941,0.884385
1,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.543329,-1.255775,1.008958
2,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.508649,-1.209911,1.070064
3,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.582959,-1.298108,1.064772
4,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,-1.518541,-1.231077,0.960003


In [57]:
tree_df.describe()

,d rambut,d medula,index medula
count,1.350000e+03,1.350000e+03,1.350000e+03
mean,-2.105312e-17,2.105312e-17,2.105312e-16
std,1.000371e+00,1.000371e+00,1.000371e+00
min,-1.615506e+00,-1.566391e+00,-3.483125e+00
25%,-1.377351e+00,-1.139947e+00,-5.480910e-01
50%,2.228681e-01,1.523961e-01,2.406108e-01
75%,6.723243e-01,7.041868e-01,6.908597e-01
max,2.432194e+00,2.953173e+00,1.583421e+00


In [58]:
# get the percentile to categorize data
bins_d_rambut = [-float('inf'), tree_df['d rambut'].quantile(0.25), tree_df['d rambut'].quantile(0.5), tree_df['d rambut'].quantile(0.75), float('inf')]
bins_d_medula = [-float('inf'), tree_df['d medula'].quantile(0.25), tree_df['d medula'].quantile(0.5), tree_df['d medula'].quantile(0.75), float('inf')]
bins_index = [-float('inf'), tree_df['index medula'].quantile(0.25), tree_df['index medula'].quantile(0.5), tree_df['index medula'].quantile(0.75), float('inf')]
labels = ['under 25%', 'under 50%', 'under 75%', '75% above']

In [59]:
# convert numerical to categorical
tree_df['d rambut'] = pd.cut(df['d rambut'], bins=bins_d_rambut, labels=labels, include_lowest=True)
tree_df['d medula'] = pd.cut(df['d medula'], bins=bins_d_medula, labels=labels, include_lowest=True)
tree_df['index medula'] = pd.cut(df['index medula'], bins=bins_index, labels=labels, include_lowest=True)

In [63]:
tree_df.head()

,spesies,color,hair cross section in the base,hair cross section in the middle,hair cross section in the tip,medulla cross section in the base,medulla cross section in the middle,medulla cross section in the tip,d rambut,d medula,index medula
0,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,under 25%,under 25%,75% above
1,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,under 25%,under 25%,75% above
2,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,under 25%,under 25%,75% above
3,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,under 25%,under 25%,75% above
4,S. barbatus,black at the base to the middle broken white a...,circular,circular,circular,flower shape,flower shape,flower shape,under 25%,under 25%,75% above


## Modelling

In [68]:
# split data for tree C5.0
y_tree = tree_df['spesies']
X_tree = tree_df.drop('spesies', axis=1)

In [71]:
train_x, test_x, train_y, test_y = train_test_split(X_tree, y_tree, test_size=0.2, stratify=y)

In [67]:
# split data into 11 folds
skf = StratifiedKFold(n_splits=11)

### C5.0

In [ ]:
model = C45Classifier()
# using folds
for train_index, test_index in skf.split(X_tree, y_tree):
  model.fit(X_tree.iloc[train_index], y_tree.iloc[train_index])
  model.evaluate(X_tree.iloc[test_index], y_tree[test_index])


In [72]:
model = C45Classifier()
model.fit(train_x, train_y)
model.evaluate(test_x, test_y)

Evaluation result: 
Total accuracy:  0.9666666666666667
Accuracy  S. scrofa :  0.9259259259259259
Accuracy  S. barbatus :  1.0
Accuracy  S. verrucossus :  1.0
Accuracy  S. celebensis :  1.0
Accuracy  S. scofa :  0.9074074074074074


In [74]:
model.generate_tree_diagram(graphviz, 'tree_C5')

'tree_C5.png'

In [75]:
model.summary()

Decision Tree Classifier Summary
Number of Instances   :  1080
Number of Attributes  :  10
Number of Leaves      :  39
Number of Rules       :  39
Tree Depth            :  4
